# PyTorch Datasets and DataLoaders
This demo covers PyTorch Datasets and DataLoaders. We will pickup and cover the topics from the video. 

# PyTorch Datasets

### PyTorch Pre-loaded Datasets
Let's begin by covering "pre-loaded" Datasets in PyTorch

These are perfect for beginning working with Datasets or for research/experimentation.

Pre-loaded Datasets available: Image, Text and Audio

In [1]:
# Let's begin with Pre-loaded Audio files
# Import torchaudio 
import torchaudio.datasets

# To get a list of available Audio Datasets go to Documentation URL: https://pytorch.org/audio/stable/datasets.html

In [ ]:
# Create a dataset using DR_VCTK (Device Recorded VCTK https://pytorch.org/audio/stable/references.html#id42)
audio_dataset = torchaudio.datasets.DR_VCTK(root='./audio', subset='test', download=True) 


In [4]:
# Lets do a pre-loaded image dataset
# Import the torchvision datasets library
import torchvision.datasets
from torchvision.transforms import ToTensor

In [5]:
# Create a dataset from the FashionMNIST classification dataset 
image_dataset = torchvision.datasets.FashionMNIST(root='./fashion', train=False, download=True, transform=ToTensor())

# NOTE: Ignore the transform for now. Tensors are required. 

In [ ]:
# Let's print the Classes of a dataset 
print(image_dataset.classes)

In [ ]:
# Print the classes to their indexes 
print(image_dataset.class_to_idx)

# This is an attribute used to map class names to integer values because models require integer values for training


In [ ]:
# Reverse the class to index mapping for plotting
class_to_index_map = image_dataset.class_to_idx
index_to_class_map = {v: k for k, v in class_to_index_map.items()}
print(index_to_class_map)

In [ ]:
# Lets get a visual of our dataset with 9 random images
import torch
import matplotlib.pyplot as plt


# Set up our plot
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(image_dataset), size=(1,)).item()
    img, label = image_dataset[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(index_to_class_map[label])
    plt.axis("off")
    plt.imshow(img.squeeze())
plt.show()

# PyTorch DataLoaders
Now that we have a working dataset, lets begin defining how we are going to present or load our data to our model.

This is done using DataLoaders!

In [12]:
# Import DataLoader
from torch.utils.data import DataLoader

In [14]:
# Create a new dataloader from our image_dataset above
image_dataloader = DataLoader(dataset=image_dataset, batch_size=64, shuffle=True, num_workers=1)

### DataLoader Parameters review
batch_size: Number of samples (images) are loaded at a time.

shuffle: When True, images are randomized before sending to the model.

num_workers: Number of processes to use for loading data. 

In [ ]:
# iterate through the image_dataloader
features, labels = next(iter(image_dataloader))
# Print the batch size and the number of labels
print(f"Features shape: {features.size()}")
print(f"Labels shape: {labels.size()}")

In [ ]:
# Show image with its label from a random image in our batch
import random

# Get a random value between 0 and our batch size
rand_indx = random.randint(0, labels.size()[0])

# set image and label
img = features[rand_indx].squeeze()
label = labels[rand_indx]

# Plot the image
plt.imshow(img)
plt.show()

# Print its label and map its numeric value to the actual name of its class
print(f"Label: {label} -> {index_to_class_map[label.item()]}")

# Review
So far we have created Datasets using PyTorch's pre-loaded datasets.

We have also created a DataLoader used to define our to present our dataset to our model.

# Custom Datasets
Lets take a look at how to create custom datasets in Pytorch using our own existing images.

In [19]:
# Import Dataset
from torch.utils.data import Dataset

In [20]:
import pandas as pd
from PIL import Image
from torchvision import transforms
from torch.utils.data import Dataset


# Define our Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, annotations_file, class_list):
        self.df = pd.read_csv(annotations_file)
        self.class_list = class_list

    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        image = Image.open(self.df.file_path[index])
        img_url = self.df.file_path[index]
        # Images must be tensors. Ignore transformations for now.
        convert_tensor = transforms.ToTensor()
        image = convert_tensor(image)
        label = self.class_list.index(self.df.label[index])

        return image, label, img_url

In [ ]:
"""
Dataset Class Review

__init__ method: Peforms initial setup and load the data. 

__len__ method: Returns the number of samples for batch.

__getitem__ method: Retrieves a single data sample based on an index.

"""

In [22]:
# Lets create our custom dataset!
# We must define an annotations file and a list of classes
class_list = ["cat", "dog"]

In [ ]:
# Create custom dataset
custom_dataset = CustomImageDataset(annotations_file='labels.csv', class_list=class_list)
print(custom_dataset)

In [ ]:
# Print attributes of our dataset (__init__ method)

# Display our annotations
print(f"Annotations data: \n{custom_dataset.df}") 

In [ ]:
# Show our classes
print(f"Classes: {custom_dataset.class_list}")

In [ ]:
# Show our class to index map
print(f"Mapped Classes: {custom_dataset.class_to_idx}")

In [28]:
# Create our own mapper OR we could add this in our __getitem__ method
custom_class_labels_map = {0: 'cat', 1: 'dog'}

In [ ]:
# Lets get a visual of our dataset with 9 random images
import torch
import matplotlib.pyplot as plt
from PIL import Image


# Set up our plot
figure = plt.figure(figsize=(8, 8))
cols, rows = 3, 3
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(custom_dataset), size=(1,)).item()
    img, label = custom_dataset[sample_idx][2], custom_dataset[sample_idx][1]
    img = Image.open(img)
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img)
plt.show()

In [31]:
# Create a DataLoader for our custom dataset
custom_dataloader = DataLoader(dataset=custom_dataset, batch_size=64, shuffle=True)

In [ ]:
# Iterate through this dataloader like we did above
features, labels, urls = next(iter(custom_dataloader))
# Print the batch size and the number of labels
print(f"Features shape: {features.size()}")
print(f"Labels shape: {labels.size()}")

In [ ]:
# Show image with its label from a random image in our batch
import random

# Get a random value between 0 and our batch size
rand_indx = random.randint(0, labels.size()[0])

# set image and label
img = urls[rand_indx]
label = labels[rand_indx]

# Plot the image
img = Image.open(img)
plt.imshow(img)
plt.show()

# Print its label and map its numeric value to the actual name of its class
print(f"Label: {label} -> {custom_class_labels_map.get(label.item())}")

# Torchvision ImageFolder
Create a dataset using the folder structure as a way to label your images.

This utility simplifies the process of loading datasets where images are organized in a directory structure.

Example:
```bash
images/
    ├── cat/
    │   ├── cat1.jpg
    │   ├── cat2.jpg
    ├── dog/
    │   ├── dog1.jpg
    │   └── dog2.jpg
```

Each image will be labled by its directory.


In [34]:
# Import torchvision
import torchvision

In [ ]:
# Create a dataset using ImageFolder 
image_folder_dataset = torchvision.datasets.ImageFolder(
    root="images", # images directory
    transform=transforms.Compose([transforms.ToTensor()])) # Ignore this for now
print(image_folder_dataset)

In [ ]:
# Let's print the Classes of a dataset 
print(image_folder_dataset.classes)

In [ ]:
# Print the classes to their indexes 
print(image_folder_dataset.class_to_idx)

In [38]:
# Load into dataloader
image_folder_dataloader = DataLoader(image_folder_dataset, batch_size=64, shuffle=True)

In [39]:
# Retrieve one batch of images and labels
images, labels = next(iter(image_folder_dataloader))


In [ ]:
# Plot the batch above
fig, axes = plt.subplots(1, len(images), figsize=(8, 8))

for i, (img, label) in enumerate(zip(images, labels)):
    img = img.permute(1, 2, 0) # Ignore this
    axes[i].imshow(img)
    axes[i].set_title(image_folder_dataset.classes[label])
    axes[i].axis("off")
plt.show()